<a href="https://colab.research.google.com/github/chetan-957/GIS/blob/main/ps4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Analysis of Electric Vehicles' Impact on PM2.5 Levels in California**
- **Study Focus**:
  - Growth of electric vehicle (EV) charging stations and electric vehicle adoption in California counties.
  - Shape file Extracted from Website: https://www.geodata.gov/
  - Analyzing the correlation between EV growth, population increases, and the total number of vehicles in each county.
  - Understanding the relationship between electric vehicle adoption, population, vehicle population, and the availability of charging infrastructure across different regions in the state.
  - Website: https://www.epa.gov/aqs
  -Data Available: Environmental and air quality data. Shapefiles for air quality monitoring stations can be found here, including PM2.5 measurements.
  -Website: https://www.arb.ca.gov/
  
  - *Data* Available: Air quality monitoring data, including geographic datasets on air quality and emissions. Some shapefiles related to California air quality monitoring may be available.
- **Objective**:
  - Predict and anticipate future growth of electric vehicles based on current trends.

- **PM2.5 Analysis**:
  - Evaluating how the rise in electric vehicles impacts PM2.5 (particulate matter) levels in the air over time.
  - Comparing areas with high EV adoption and charging infrastructure to those with lower adoption to assess reductions in air pollution.
  - Investigating the potential for electric vehicles to improve air quality and public health by lowering PM2.5 concentrations in California.
<br>
<br>
<br>
<br>
<br>
Done by Sai Srija Ganapuram and Injavarapu Chetan Sai Abhishek

In [ ]:
%%capture
!pip install geopandas==1.0.1
!pip install mapclassify
!pip install reverse_geocoder
!pip install contextily
!pip install ipywidgets
!pip install ipyleaflet
!pip install folium geopandas
!pip install folium branca

In [ ]:
# All Imports
import os, zipfile #basics
import pandas as pd #data management
import matplotlib.pyplot as plt #vis
import csv #to read csv files
from shapely.geometry import Point
import reverse_geocoder as rg
import numpy as np
import seaborn as sns
import folium
from folium.plugins import MarkerCluster, TimestampedGeoJson, HeatMap
from folium import IFrame
from datetime import datetime
from IPython.display import display, IFrame
from IPython.display import Image,HTML

import geopandas as gpd #gis/maps: a sister of pandas; does the job;

import mapclassify #need for thematic map classification
import ipywidgets as widgets
from ipyleaflet import Map, basemap_to_tiles, GeoJSON
from ipywidgets import interact
from shapely.geometry import Point

#will display all output not just last command
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from google.colab import files #to download from colab onto hd

from google.colab import data_table
data_table.enable_dataframe_formatter()
import matplotlib.colors as mcolors
from sklearn.preprocessing import KBinsDiscretizer
import contextily as ctx

In [ ]:
gpd.__version__

In [ ]:
! wget -q -O ca_counties.zip https://github.com/sg2083/GIS_repo/raw/main/ca_counties.zip
zip_ref = zipfile.ZipFile('ca_counties.zip', 'r')
zip_ref.extractall('California_Shape_Files') # Extract all files to a specific directory

CAl0=gpd.read_file('California_Shape_Files/CA_Counties.shp') #load the shapefile with gpd as tlC
zip_ref.close() # Close the zip file after reading the shapefile
CAl0.plot(color='gold', edgecolor='black') # Change color to lightgreen, edgecolor to black
plt.show()


In [ ]:
# Checking for the data types and checking the values of the data
CAl0.dtypes
CAl0[CAl0.columns.difference(['geometry'])].head(2)

# Selecting only Few Variables for the Table
CAl=CAl0[['NAME','NAMELSAD','Shape_Area','geometry']]

### **Exploring and Manipulating Population Data Of California**
taking a text file of California's population data and manipulating it to fit the structure of a pre-selected table. The goal is to ***clean, format, and align the data*** for further analysis.
Then we'll plot the latest data on the Shape File of California.

In [ ]:
#Data cleaning and merging for plotting
# Define the input file path to get the population value
! wget -q -O population_data.txt https://github.com/sg2083/GIS_repo/raw/main/population_data.txt
input_file = 'population_data.txt'

# Read the CSV data from the text file
data = []
with open(input_file, 'r') as txt_file:
    csv_reader = csv.reader(txt_file)
    headers = next(csv_reader)  # Skip the header
    for row in csv_reader:
        data.append(row)

# Convert the data into a DataFrame
df = pd.DataFrame(data, columns=['Year', 'Period', 'Area', 'Population Source', 'Population'])

# Convert Population to numeric
df['Population'] = pd.to_numeric(df['Population'].str.replace(",", ""))

# Population_of_california  the DataFrame to restructure it
POP_CAL_pivot_df = df.pivot_table(index='Area', columns='Year', values='Population')

# Rename columns for clarity (optional)
POP_CAL_pivot_df.columns = [f'Year-{col}' for col in POP_CAL_pivot_df.columns]

# Reset index to make 'Area' a column again
POP_CAL_pivot_df = POP_CAL_pivot_df.reset_index()

# Print the result
print(POP_CAL_pivot_df.head(2))

# Optionally, save the result to a CSV file
POP_CAL_pivot_df.to_csv('year-wise.csv', index=False)


After refining the dataset, we integrate it with shapefile data to create an engaging thematic map, showcasing the population distribution for 2022.

In [ ]:
# Plotting Population Data on California Map By Merging it on Basis of County
after_merge=CAl.merge(POP_CAL_pivot_df, left_on='NAMELSAD', right_on='Area',how='left')
# fig, ax = plt.subplots(1, figsize=(20, 10))


## **Plotting the population density**
Plotting population data on a map allows for a visual representation of demographic distribution across a given geographic area, providing valuable insights into population density, trends, and spatial patterns.

In [ ]:
! wget -q -O County_Industry_Data.csv https://github.com/sg2083/GIS_repo/raw/main/County_Industry_Data.csv

df_industry = pd.read_csv('County_Industry_Data.csv')
df_industry_filter = df_industry[df_industry['ST_ABBREV'] == 'CA']
get_df_industry = df_industry_filter[['ID','NAME','INDTECH_CY','INDEDUC_CY']]

after_merge_industry=after_merge.merge(get_df_industry, left_on='NAMELSAD', right_on='NAME',how='left')
after_merge_industry = after_merge_industry.drop(columns=['Area', 'NAME_y'])
print(after_merge_industry.head(2))

fig, ax = plt.subplots(1, figsize=(16, 10))

after_merge_industry.plot(ax=ax,
         column='INDTECH_CY',        # Change this to any other year if needed (e.g., 'Year-2007', 'Year-1995')
         legend=True,
         cmap='YlOrBr',             # Yellow to Red color gradient
         scheme='natural_breaks',    # Classify data into natural breaks (Jenks)
         k=5,                      # Number of classes
         vmin=0.1 ,
         edgecolor='grey',           # Border color for counties
         linewidth=2,                # Line width for borders
         legend_kwds={               # Customize the legend
             "fmt": "{:,.0f}",       # Format numbers with commas and no decimal points
             'loc': 'lower right',   # Legend location
             'title_fontsize': 'medium',  # Title font size for the legend
             'fontsize': 'small',         # Font size for the legend
             'markerscale': 1.4           # Scale of the markers in the legend
         })

ax.title.set_text("California County Industry Data Tech Services in 2018")

ax.set_xticks([])
ax.set_yticks([])




## **Mapping Electric Vehicle Charging Stations**
We obtained electric vehicle data that needed cleaning. Although the exact location of the charging stations wasn’t provided, we had longitude and latitude coordinates. Using this, we identified the county for each station and added that information to the data. After merging the longitude and latitude values to create the geometric points, we successfully plotted the EV stations on the map.

In [ ]:
# Reading EV Charging Station Data
! wget -q -O ev_data.csv https://github.com/sg2083/GIS_repo/raw/main/ev_data.csv
ev_data = pd.read_csv('ev_data.csv')
ev_data.head(2)

In [ ]:
# Function to get county name
def get_county_from_lat_lon(lat, lon):
    coordinates = (lat, lon)
    result = rg.search(coordinates)  # Returns a list of dictionaries
    return result[0]['admin2']

# Apply the function to each row
for index, row in ev_data.iterrows():
    ev_data.loc[index, 'County'] = get_county_from_lat_lon(row['Latitude'], row['Longitude'])

#  Updated ev_data with county information
ev_data.to_csv('ev_data_with_county.csv', index=False)


In [ ]:
ev_data_with_county = pd.read_csv('ev_data_with_county.csv')
ev_data_with_county.head(3)

In [ ]:
# Create a GeoDataFrame with geometry for EV stations
ev_gdf = gpd.GeoDataFrame(
    ev_data_with_county,
    geometry=gpd.points_from_xy(ev_data_with_county.Longitude, ev_data_with_county.Latitude)
)

# Explicitly set the CRS of ev_gdf to match tlC0 - replace 'epsg:xxxx' with the actual EPSG code
ev_gdf = ev_gdf.set_crs("epsg:4326")
after_merge = after_merge.to_crs("epsg:4326")


merged_data = after_merge.merge(ev_data_with_county, left_on='NAMELSAD', right_on='County', how='left')


## **Analyzing Electric Vehicle Distribution and Charging Station Accessibility in California**
We analyzed EV data across California and visualized it. Next, we layered the locations of charging stations and EVs on the map to see if the stations are placed where they’re most needed.

In [ ]:
# Load the vehicle population data from the Excel file
! wget -q -O vehicle_pop.xlsx https://github.com/sg2083/GIS_repo/raw/main/vehicle_pop.xlsx
vehicle_pop = pd.read_excel('vehicle_pop.xlsx', sheet_name='County')

# Define a reusable function to filter, group, and pivot data for a given year
def process_vehicle_data(df, year):
    # Filter data by year and fuel types
    filtered_df = df[(df['Data Year'] == year) &
                     (df['Fuel Type'].isin(['Battery Electric (BEV)', 'Plug-in Hybrid (PHEV)']))]

    # Group by county and fuel type and sum the vehicle counts
    county_fuel_count = filtered_df.groupby(['County', 'Fuel Type'])['Number of Vehicles'].sum().reset_index()

    # Pivot the table for better readability
    county_fuel_pivot = county_fuel_count.pivot(index='County', columns='Fuel Type', values='Number of Vehicles').reset_index()

    # Fill NaN values with 0
    county_fuel_pivot = county_fuel_pivot.fillna(0)

    return county_fuel_pivot

# Process data for each year
county_fuel_2023 = process_vehicle_data(vehicle_pop, 2023)
county_fuel_2010 = process_vehicle_data(vehicle_pop, 2010)
county_fuel_2017 = process_vehicle_data(vehicle_pop, 2017)

# Add a new column 'Total' for the sum of BEV and PHEV for each year
county_fuel_2023['Total_EV_Vehicles_2023'] = (county_fuel_2023['Battery Electric (BEV)'] +
                                              county_fuel_2023['Plug-in Hybrid (PHEV)'])

county_fuel_2010['Total_EV_Vehicles_2010'] = (county_fuel_2010['Battery Electric (BEV)'] +
                                              county_fuel_2010['Plug-in Hybrid (PHEV)'])

county_fuel_2017['Total_EV_Vehicles_2017'] = (county_fuel_2017['Battery Electric (BEV)'] +
                                              county_fuel_2017['Plug-in Hybrid (PHEV)'])

# Merge the data from all years into a single DataFrame
county_fuel_combined = county_fuel_2023[['County', 'Total_EV_Vehicles_2023']].merge(
    county_fuel_2010[['County', 'Total_EV_Vehicles_2010']], on='County', how='left').merge(
    county_fuel_2017[['County', 'Total_EV_Vehicles_2017']], on='County', how='left')

# Replace any remaining NaN values with 0
county_fuel_combined = county_fuel_combined.fillna(0)

# Convert only numeric columns to integers, excluding 'County'
numeric_columns = county_fuel_combined.select_dtypes(include=['float', 'int']).columns
county_fuel_combined[numeric_columns] = county_fuel_combined[numeric_columns].astype(int)

# Print the result
county_fuel_combined[['Total_EV_Vehicles_2010', 'Total_EV_Vehicles_2017', 'Total_EV_Vehicles_2023']].head(3)


After filtering the dataset to focus on the counts of electric vehicles (EVs) for the years 2010, 2017, and 2023, I aimed to visualize the growth in EV adoption over this period. By creating side-by-side plots for each of these years, I intended to provide a clear comparison that highlights the increase in vehicle counts. This approach allows for a straightforward visual analysis, making it easier to observe trends and changes in EV numbers over time. The resulting plots serve as an effective tool to illustrate the progress in electric vehicle usage, reflecting broader trends in sustainability and transportation technology.

In [ ]:
# Merging the datasets
merged_data = after_merge.merge(county_fuel_combined, left_on='NAME', right_on='County', how='left')

# Find the overall min and max values for color scale consistency
vmin = merged_data[['Total_EV_Vehicles_2010', 'Total_EV_Vehicles_2017', 'Total_EV_Vehicles_2023']].min().min()
vmax = merged_data[['Total_EV_Vehicles_2010', 'Total_EV_Vehicles_2017', 'Total_EV_Vehicles_2023']].max().max()

# Create a figure with 3 subplots
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot for 2010
merged_data.plot(column='Total_EV_Vehicles_2010',
                 cmap='GnBu',
                 linewidth=0.8,
                 edgecolor='0.8',
                 legend=True,
                 ax=axes[0],
                 vmin=vmin,  # Consistent color scale
                 vmax=vmax,  # Consistent color scale
                 missing_kwds={'color': 'lightgrey'})
axes[0].set_title('Total BEV and PHEV Vehicles by County (2010)')
axes[0].set_axis_off()  # Turn off the axis for better map presentation

# Plot for 2017
merged_data.plot(column='Total_EV_Vehicles_2017',
                 cmap='GnBu',
                 linewidth=0.8,
                 edgecolor='0.8',
                 legend=True,
                 ax=axes[1],
                 vmin=vmin,
                 vmax=vmax,
                 missing_kwds={'color': 'lightgrey'})
axes[1].set_title('Total BEV and PHEV Vehicles by County (2017)')
axes[1].set_axis_off()

# Plot for 2023
merged_data.plot(column='Total_EV_Vehicles_2023',
                 cmap='GnBu',
                 linewidth=0.8,
                 edgecolor='0.8',
                 legend=True,
                 ax=axes[2],
                 vmin=vmin,
                 vmax=vmax,
                 missing_kwds={'color': 'lightgrey'})
axes[2].set_title('Total BEV and PHEV Vehicles by County (2023)')
axes[2].set_axis_off()

plt.tight_layout()  # Adjust spacing between subplots
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 3))
merged_data.plot(column='Total_EV_Vehicles_2023',
                cmap='GnBu',
                linewidth=0.8,
                edgecolor='0.8',
                legend=True,
                ax=ax,
                missing_kwds={'color': 'lightgrey'})

ev_gdf.plot(ax=ax, marker='o', color='Orange', markersize=0.5, alpha=0.5)

ax.set_title('Total BEV and PHEV Vehicles by County and EV Charging Stations Available')
plt.show()


In [ ]:
# Assuming 'merged_data' is your GeoDataFrame with county boundaries and EV data
merged_data['centroid'] = merged_data.geometry.centroid

# Print the updated GeoDataFrame
print(merged_data.head(2))  # Display the first two rows to verify centroid calculation


we are visualizing electric vehicle (EV) data for the year 2023 by plotting county boundaries and their centroids on a map. First, we create a figure and axis for our plot, then render the county boundaries in beige with black edges for clarity. Next, we define a custom colormap for better visual representation of the EV data, specifically adjusting it to start from a lighter shade. We then plot the centroids of each county, using the total number of EVs from 2017 to determine the color intensity of the markers, making it easier to identify areas with higher vehicle counts. The plot is enhanced with a title and labeled axes, ultimately providing insights into the distribution of EVs across different regions. This visual analysis helps us observe patterns in EV adoption over time.

In [ ]:
# fig, ax = plt.subplots(figsize=(5, 5))

# # Plot the county boundaries with a specific color
# merged_data.plot(ax=ax, color='beige', edgecolor='black', linewidth=0.5)

# # Plot the centroids
# merged_data.plot(ax=ax, column='Total_EV_Vehicles', marker='o', markersize=10, cmap='Reds', legend=True, scheme='natural_breaks')

# Create a custom colormap with a modified minimum color
cmap = plt.cm.get_cmap('Reds')  # Get the 'Reds' colormap
new_cmap = cmap(np.linspace(0.2, 1, cmap.N))  # Start from 0.2 instead of 0
new_cmap = mcolors.ListedColormap(new_cmap)  # Create a new colormap



#**Air Quality Index of 2010 , 2017 and 2023**
This code starts by downloading air quality data for the years 2010, 2017, and 2023 from specific online sources. Once the data is in hand, it defines a function that processes this information: it reads each CSV file, groups the data by county, calculates the average daily PM2.5 concentrations and Air Quality Index (AQI) values, and renames the columns to make them clearer. After processing the data for all three years, the code merges these datasets based on the 'County' column, resulting in a comprehensive table that captures air quality metrics for each county across the different years.

Following that, the code combines this air quality data with existing electric vehicle (EV) information. This merging allows us to analyze how air quality relates to the number of electric vehicles in each county. To visualize this relationship, the code creates histograms showing the distribution of AQI levels for each year. Each histogram represents the frequency of counties at different AQI levels, using distinct colors to make the data easy to interpret. This visual approach helps us spot trends and patterns in air quality over time, enhancing our understanding of how air pollution might connect with the adoption of electric vehicles. Finally, the layout of the plots is adjusted for a polished presentation, and the histograms are displayed for viewing.

In [ ]:
! wget -q -O air_quality_PM_2010.csv https://github.com/sg2083/GIS_repo/raw/main/air_quality_PM_2010.csv
! wget -q -O air_quality_PM_2017.csv https://github.com/sg2083/GIS_repo/raw/main/air_quality_PM_2017.csv
! wget -q -O air_quality_PM_2023.csv https://github.com/sg2083/GIS_repo/raw/main/air_quality_PM_2023.csv
data = pd.read_csv('air_quality_PM_2010.csv')

# Function to load, group by 'County', calculate mean, round values, and rename columns
def process_data(file_path, year):
    data = pd.read_csv(file_path)
    county_grouped = data.groupby('County').mean(numeric_only=True).round(2)
    county_grouped = county_grouped.rename(columns={
        'Daily Mean PM2.5 Concentration': f'Mean PM2.5_{year}',
        'Daily AQI Value': f'Mean AQI_{year}'
    })
    return county_grouped

# Process data for 2010, 2017, and 2023
county_grouped_2010 = process_data('air_quality_PM_2010.csv', 2010)
county_grouped_2017 = process_data('air_quality_PM_2017.csv', 2017)
county_grouped_2023 = process_data('air_quality_PM_2023.csv', 2023)

# Merge data for all years on 'County'
merged_air_data = county_grouped_2010[['Site ID', f'Mean PM2.5_2010', f'Mean AQI_2010', 'Site Latitude', 'Site Longitude']].merge(
    county_grouped_2017[[f'Mean PM2.5_2017', f'Mean AQI_2017']], on='County', how='inner'
).merge(
    county_grouped_2023[[f'Mean PM2.5_2023', f'Mean AQI_2023']], on='County', how='inner'
)

# Display the merged table with data from 2010, 2017, and 2023
merged_air_data.head()

In [ ]:
# Merge EV data with the previously merged air quality data on 'County'
merged_ev_data = merged_air_data.merge(merged_data[['County', 'Total_EV_Vehicles_2023', 'Total_EV_Vehicles_2017', 'Total_EV_Vehicles_2010']], on='County', how='inner')

# Display the merged data with EV population
merged_ev_data.head()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(10, 5))

# Plot histogram for AQI levels in 2010 and 2017 on the left
axes[0].hist(merged_ev_data['Mean AQI_2010'], bins=30, alpha=0.7, label='AQI 2010', color='#ffeb3b')  # Yellow
axes[0].set_xlabel('AQI Levels')
axes[0].set_ylabel('Frequency of Counties')
axes[0].set_title('AQI Levels Distribution (2010)')
axes[0].legend()

# Plot histogram for AQI levels in 2017 and 2023 on the right
axes[1].hist(merged_ev_data['Mean AQI_2017'], bins=30, alpha=0.7, label='AQI 2017', color='#e91e63')  # Pink
axes[1].set_xlabel('AQI Levels')
axes[1].set_ylabel('Frequency of Counties')
axes[1].set_title('AQI Levels Distribution (2017)')
axes[1].legend()

axes[2].hist(merged_ev_data['Mean AQI_2023'], bins=30, alpha=0.7, label='AQI 2023', color='#388e3c')  # Dark Green
axes[2].set_xlabel('AQI Levels')
axes[2].set_ylabel('Frequency of Counties')
axes[2].set_title('AQI Levels Distribution (2023)')
axes[2].legend()
# Adjust layout to prevent overlapping
plt.tight_layout()

# Show the plot
plt.show()

derive PM2.5 pollutant from the data and plotting it on the graph. Cuz we are studying about EV vechiles and PM2.5 is related to vechile emmision pollutatant that has to be reduced  

#**Analysis of air Pollutents related to vechicles**
pollution and PM2.5 pollunent data
ChatGPT said:
ChatGPT
As a next step, we focus on visualizing PM2.5 pollutant levels to understand their relationship with electric vehicle (EV) emissions. PM2.5, fine particulate matter that can penetrate deep into the lungs, is a significant air pollutant associated with various health issues and environmental concerns. Monitoring its levels is crucial, especially as we study the impact of vehicle emissions on air quality. This plot displays the mean PM2.5 concentrations across different counties for the years 2010, 2017, and 2023. Using line plots for each year, distinct colors enhance clarity. The x-axis represents the counties, while the y-axis shows the mean PM2.5 concentrations. The visualization includes a title, axis labels, and a legend for easy interpretation. By illustrating trends in PM2.5 levels over time, this analysis highlights the importance of reducing vehicle emissions to improve air quality and public heal

In [ ]:
# Set the figure size
plt.figure(figsize=(14, 8))

# Plot each year's PM2.5 values
plt.plot(merged_ev_data['County'], merged_ev_data['Mean PM2.5_2010'], marker='o', label='Mean PM2.5 2010', color='blue')
plt.plot(merged_ev_data['County'], merged_ev_data['Mean PM2.5_2017'], marker='o', label='Mean PM2.5 2017', color='orange')
plt.plot(merged_ev_data['County'], merged_ev_data['Mean PM2.5_2023'], marker='o', label='Mean PM2.5 2023', color='green')

# Title and labels
plt.title('Mean PM2.5 Concentration Across Counties (2010, 2017, 2023)', fontsize=16)
plt.xlabel('County', fontsize=14)
plt.ylabel('Mean PM2.5 Concentration', fontsize=14)
plt.xticks(rotation=90)  # Rotate x-axis labels for better readability
plt.legend()
plt.grid()

# Show the plot
plt.tight_layout()
plt.show()

In this next step, we create a GeoDataFrame to visualize PM2.5 pollution data using latitude and longitude coordinates. This involves constructing geometrical points for each site based on their respective coordinates and ensuring the GeoDataFrame is aligned with the correct coordinate reference system (EPSG:4326). After that, we merge this PM2.5 data with a shapefile of California counties, allowing us to map the pollution levels accurately.

Now, we study PM2.5 over the three years (2010, 2017, and 2023) and plot them on an interactive map. We define a function to visualize the PM2.5 levels for each year, using a color gradient to represent different levels of pollution. This function also adds a basemap for better context.

To enhance user interaction, an interactive slider is implemented, enabling users to choose the year they wish to explore. Additionally, we employ the Folium library to create a dynamic map that visualizes Air Quality Index (AQI) values at various locations, with markers color-coded based on the AQI level. Each marker displays relevant information about the county and AQI value when clicked.

This approach not only helps in analyzing PM2.5 data over time but also emphasizes the relationship between vehicle emissions and air quality, providing insights into the effectiveness of policies aimed at reducing pollution.

In [ ]:
# Step 3: Create a GeoDataFrame using PM2.5 data (using Lat/Lon)
geometry = [Point(xy) for xy in zip(merged_ev_data['Site Longitude'], merged_ev_data['Site Latitude'])]
gdf = gpd.GeoDataFrame(merged_ev_data, geometry=geometry)

# Ensure the GeoDataFrame is using the same CRS as the shapefile (EPSG:4326 for lat/lon)
gdf.set_crs(epsg=4326, inplace=True).head(2)

# Step 4: Merge PM2.5 data with California shapefile based on county name
gdf_county = gdf.merge(merged_ev_data, left_on='County', right_on='County')

# Define a function to plot the PM2.5 data with time slider and basemap
def plot_map(year):
    # Map the PM2.5 values based on the selected year
    if year == 2010:
        gdf_county.plot(column='Mean PM2.5_2010_x', cmap='coolwarm', figsize=(6, 6), legend=True)
        plt.title('Mean PM2.5 Levels (2010)')
    elif year == 2017:
        gdf_county.plot(column='Mean PM2.5_2017_x', cmap='coolwarm', figsize=(6, 6), legend=True)
        plt.title('Mean PM2.5 Levels (2017)')
    elif year == 2023:
        gdf_county.plot(column='Mean PM2.5_2023_x', cmap='coolwarm', figsize=(6, 6), legend=True)
        plt.title('Mean PM2.5 Levels (2023)')

    # Add basemap using contextily
    ctx.add_basemap(plt.gca(), crs=gdf_county.crs.to_string(), source=ctx.providers.OpenStreetMap.Mapnik)
    plt.show()

# Step 6: Interactive map with time slider
interact(plot_map, year=widgets.IntSlider(min=2010, max=2023, step=7, value=2010))



A heat map of PM2.5 levels visually represents the concentration of particulate matter (PM2.5) across a geographic area, using colors to indicate varying levels of pollution. In the map, areas with higher PM2.5 concentrations are typically shaded in darker or more intense colors, while regions with lower concentrations appear in lighter shades. This type of map allows for a quick understanding of pollution hotspots, helping to identify areas with significant air quality issues. By analyzing a heat map of PM2.5 levels, we can assess the spatial distribution of air pollution, detect trends over time, and make informed decisions about environmental policies, urban planning, and public health interventions. It is particularly useful for highlighting the impact of vehicle emissions, industrial activities, or the effectiveness of air quality improvement measures.

In [ ]:
# Heat Map
# Create the base map centered at California's mean latitude and longitude
map_center = [merged_ev_data['Site Latitude'].mean(), merged_ev_data['Site Longitude'].mean()]
map_pm25 = folium.Map(location=map_center, zoom_start=6, width=800, height=600)

# Function to create a heatmap for a given year (2010, 2017, 2023)
def create_heatmap_for_year(data, year_column, year_name):
    # Prepare the data for the heatmap
    # Each point in the heatmap is a tuple of (latitude, longitude, PM2.5 value)
    heat_data = [
        [row['Site Latitude'], row['Site Longitude'], row[year_column]]
        for index, row in data.iterrows()
        if not pd.isnull(row[year_column])  # Ignore rows where PM2.5 value is NaN
    ]

    # Create a heatmap with PM2.5 values as the weight
    heatmap = HeatMap(heat_data, min_opacity=0.2, max_zoom=15, radius=25, blur=15, max_val=max(data[year_column]))

    # Create a separate map layer for each year
    heatmap.add_to(folium.FeatureGroup(name=f"PM2.5 {year_name}").add_to(map_pm25))

# Generate heatmaps for each year
create_heatmap_for_year(merged_ev_data, 'Mean PM2.5_2010', '2010')
create_heatmap_for_year(merged_ev_data, 'Mean PM2.5_2017', '2017')
create_heatmap_for_year(merged_ev_data, 'Mean PM2.5_2023', '2023')

# Add a layer control to toggle between the different years
folium.LayerControl().add_to(map_pm25)

# Display the map
map_pm25

## **Bivariate Analysis of Electric Vehicle Adoption and PM2.5 Levels Over Time**
In the next phase of our analysis, we will create a bivariate graph to explore the relationship between electric vehicle (EV) adoption and PM2.5 levels over time. This graph will allow us to visually examine how the growth of EVs correlates with changes in PM2.5 concentrations across different periods. By plotting both variables on the same graph, we aim to identify any trends or patterns, such as whether increased EV adoption leads to a noticeable reduction in PM2.5 levels. The bivariate graph will help us understand the potential environmental impact of EVs in reducing air pollution, providing a clearer view of the long-term effects of EV adoption on air quality. This analysis will be crucial for assessing the effectiveness of EVs in improving urban air quality and public health.

In [ ]:

# Define number of quantiles for EV and PM2.5
n_quantiles = 3
gdf_county.head(2)
# Apply quantile binning for both datasets
ev_binner = KBinsDiscretizer(n_bins=n_quantiles, encode='ordinal', strategy='quantile')
pm25_binner = KBinsDiscretizer(n_bins=n_quantiles, encode='ordinal', strategy='quantile')

# Add classified data to DataFrame
gdf_county['ev_class'] = ev_binner.fit_transform(gdf_county[['Total_EV_Vehicles_2023_y']]).astype(int)
gdf_county['pm25_class'] = pm25_binner.fit_transform(gdf_county[['Mean PM2.5_2023_x']]).astype(int)

# Create bivariate class
gdf_county['bivariate_class'] = gdf_county['ev_class'] * n_quantiles + gdf_county['pm25_class']

# Define a color dictionary for the 9 possible combinations (3 EV classes * 3 PM2.5 classes)
bivariate_colors = {
    0: '#e8e8e8',  # low EV, low PM2.5
    1: '#ace4e4',  # low EV, medium PM2.5
    2: '#5ac8c8',  # low EV, high PM2.5
    3: '#dfb0d6',  # medium EV, low PM2.5
    4: '#a5add3',  # medium EV, medium PM2.5
    5: '#5698b9',  # medium EV, high PM2.5
    6: '#be64ac',  # high EV, low PM2.5
    7: '#8c62aa',  # high EV, medium PM2.5
    8: '#3b4994'   # high EV, high PM2.5
}
gdf_county.rename(columns={'geometry': 'centro'}, inplace=True)
gdf_county = gdf_county.merge(merged_data[['County','geometry']], on='County', how='inner')

gdf = gpd.GeoDataFrame(gdf_county, geometry=gdf_county['geometry'])

# Create a map centered on the USA (adjust based on your location)
# m = folium.Map(location=[37.8, -96], zoom_start=4)

# # Add polygons with bivariate colors based on the 'bivariate_class'
# for _, row in gdf.iterrows():
#     folium.GeoJson(
#         row['geometry'],
#         style_function=lambda feature, row=row: {
#             'fillColor': bivariate_colors[row['bivariate_class']],
#             'color': 'black',  # Boundary color
#             'weight': 0.5,
#             'fillOpacity': 0.7,
#         }
#     ).add_to(m)

# Add a legend manually (you can format this as needed)
legend_html = '''
<div style="position: fixed;
     bottom: 20px; left: 50px; width: 200px; height: 270px;
     background-color: white; z-index:9999; font-size:14px;">
     <b>&nbsp; Bivariate Legend</b><br>
     &nbsp; EV ↓ PM2.5 →<br>
     &nbsp; <i style="background-color: #e8e8e8; width: 20px; height: 20px; display: inline-block;"></i>&nbsp; Low EV, Low PM2.5<br>
     &nbsp; <i style="background-color: #ace4e4; width: 20px; height: 20px; display: inline-block;"></i>&nbsp; Low EV, Med PM2.5<br>
     &nbsp; <i style="background-color: #5ac8c8; width: 20px; height: 20px; display: inline-block;"></i>&nbsp; Low EV, High PM2.5<br>
     &nbsp; <i style="background-color: #dfb0d6; width: 20px; height: 20px; display: inline-block;"></i>&nbsp; Med EV, Low PM2.5<br>
     &nbsp; <i style="background-color: #a5add3; width: 20px; height: 20px; display: inline-block;"></i>&nbsp; Med EV, Med PM2.5<br>
     &nbsp; <i style="background-color: #5698b9; width: 20px; height: 20px; display: inline-block;"></i>&nbsp; Med EV, High PM2.5<br>
     &nbsp; <i style="background-color: #be64ac; width: 20px; height: 20px; display: inline-block;"></i>&nbsp; High EV, Low PM2.5<br>
     &nbsp; <i style="background-color: #8c62aa; width: 20px; height: 20px; display: inline-block;"></i>&nbsp; High EV, Med PM2.5<br>
     &nbsp; <i style="background-color: #3b4994; width: 20px; height: 20px; display: inline-block;"></i>&nbsp; High EV, High PM2.5<br>
</div>
'''

# m.get_root().html.add_child(folium.Element(legend_html))

# Display the map
# m


In [ ]:
import geopandas as gpd
import folium
from sklearn.preprocessing import KBinsDiscretizer
from IPython.display import display, HTML

# Function to create bivariate map for a specific year
def create_bivariate_map(gdf_county, year, n_quantiles=3):
    # Define columns based on year
    ev_column = f'Total_EV_Vehicles_{year}_y'
    pm25_column = f'Mean PM2.5_{year}_x'

    # Apply quantile binning for EV and PM2.5 data
    ev_binner = KBinsDiscretizer(n_bins=n_quantiles, encode='ordinal', strategy='quantile')
    pm25_binner = KBinsDiscretizer(n_bins=n_quantiles, encode='ordinal', strategy='quantile')

    # Add classified data to DataFrame
    gdf_county['ev_class'] = ev_binner.fit_transform(gdf_county[[ev_column]]).astype(int)
    gdf_county['pm25_class'] = pm25_binner.fit_transform(gdf_county[[pm25_column]]).astype(int)

    # Create bivariate class
    gdf_county['bivariate_class'] = gdf_county['ev_class'] * n_quantiles + gdf_county['pm25_class']

    # Define color mapping
    bivariate_colors = {
        0: '#e8e8e8', 1: '#ace4e4', 2: '#5ac8c8',
        3: '#dfb0d6', 4: '#a5add3', 5: '#5698b9',
        6: '#be64ac', 7: '#8c62aa', 8: '#3b4994'
    }

    # Create folium map
    m = folium.Map(location=[37.8, -96], zoom_start=4)

    # Add polygons with bivariate colors
    for _, row in gdf_county.iterrows():
        folium.GeoJson(
            row['geometry'],
            style_function=lambda feature, row=row: {
                'fillColor': bivariate_colors[row['bivariate_class']],
                'color': 'black',
                'weight': 0.5,
                'fillOpacity': 0.7,
            }
        ).add_to(m)

    return m

# Generate maps for each year
map_2010 = create_bivariate_map(gdf_county, 2010)
map_2017 = create_bivariate_map(gdf_county, 2017)
map_2023 = create_bivariate_map(gdf_county, 2023)

# Define a standalone legend HTML
legend_html = '''
<div style="position: relative;
     margin-top: 20px; width: 200px; height: 270px;
     background-color: white; z-index:9999; font-size:14px; border: 1px solid black;
     padding: 10px;">
     <b>&nbsp; Bivariate Legend</b><br>
     &nbsp; EV ↓ PM2.5 →<br>
     &nbsp; <i style="background-color: #e8e8e8; width: 20px; height: 20px; display: inline-block;"></i>&nbsp; Low EV, Low PM2.5<br>
     &nbsp; <i style="background-color: #ace4e4; width: 20px; height: 20px; display: inline-block;"></i>&nbsp; Low EV, Med PM2.5<br>
     &nbsp; <i style="background-color: #5ac8c8; width: 20px; height: 20px; display: inline-block;"></i>&nbsp; Low EV, High PM2.5<br>
     &nbsp; <i style="background-color: #dfb0d6; width: 20px; height: 20px; display: inline-block;"></i>&nbsp; Med EV, Low PM2.5<br>
     &nbsp; <i style="background-color: #a5add3; width: 20px; height: 20px; display: inline-block;"></i>&nbsp; Med EV, Med PM2.5<br>
     &nbsp; <i style="background-color: #5698b9; width: 20px; height: 20px; display: inline-block;"></i>&nbsp; Med EV, High PM2.5<br>
     &nbsp; <i style="background-color: #be64ac; width: 20px; height: 20px; display: inline-block;"></i>&nbsp; High EV, Low PM2.5<br>
     &nbsp; <i style="background-color: #8c62aa; width: 20px; height: 20px; display: inline-block;"></i>&nbsp; High EV, Med PM2.5<br>
     &nbsp; <i style="background-color: #3b4994; width: 20px; height: 20px; display: inline-block;"></i>&nbsp; High EV, High PM2.5<br>
</div>
'''

# Display maps side by side and the legend below in Colab
display(HTML(f'''
<div style="display: flex; justify-content: space-around; margin-bottom: 20px;">
    <div style="width: 32%;">{map_2010._repr_html_()}</div>
    <div style="width: 32%;">{map_2017._repr_html_()}</div>
    <div style="width: 32%;">{map_2023._repr_html_()}</div>
</div>
<div style="display: flex; justify-content: center;">
    {legend_html}
</div>
'''))


### Clustering Analysis of EV Adoption and PM2.5 Levels

In this analysis, we are using clustering techniques to better understand the relationship between electric vehicle (EV) adoption and PM2.5 levels over time. The previous bivariate maps were difficult to interpret due to the complexity of the data, so clustering provides a more accessible way to group regions with similar patterns. By applying **K-means clustering**, we aim to categorize counties based on their PM2.5 levels and EV adoption rates over three time points (2010, 2017, and 2023).

The steps in the code involve:

1. **Standardizing the Data**: We begin by standardizing the relevant features (PM2.5 levels and EV adoption data) to ensure they are on a comparable scale.
2. **Applying K-means Clustering**: We use the K-means algorithm with 5 clusters to group the counties based on the patterns observed in the data. Each cluster represents a distinct combination of PM2.5 trends and EV adoption levels.
3. **Assigning Cluster Descriptions**: Each cluster is given a descriptive label based on the general characteristics of the data, such as "Moderate Improvement in PM2.5 with Moderate EV Adoption" or "Stable PM2.5 with Slow EV Growth."
4. **Visualizing the Clusters**: Finally, we visualize these clusters on a **Folium map** where each county is color-coded according to its assigned cluster. The map includes interactive popups showing detailed information about the PM2.5 levels and EV adoption in each region, making it easier to compare counties across different clusters.

The clustering approach helps us simplify the complexity of the data, providing a clearer view of how EV adoption is potentially influencing PM2.5 levels over time across different counties. The resulting map offers a more accessible and interpretable representation of the data, allowing us to identify trends and regional variations more effectively.

In [ ]:
import pandas as pd
import geopandas as gpd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import folium

# Assuming `gdf` is your GeoDataFrame with the relevant columns
# Select relevant columns for clustering
data_for_clustering = gdf_county[['Mean PM2.5_2010_x', 'Mean PM2.5_2017_x', 'Mean PM2.5_2023_x',
                           'Total_EV_Vehicles_2010_y', 'Total_EV_Vehicles_2017_y', 'Total_EV_Vehicles_2023_y']]

# Step 1: Standardize the data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_for_clustering)

# Step 2: Apply K-means with 5 clusters
kmeans = KMeans(n_clusters=5, random_state=0)
gdf['Cluster'] = kmeans.fit_predict(data_scaled)

# Step 3: Define descriptions and colors for each cluster
cluster_descriptions = {
    0: "Cluster 0 - Slow improvement PM2.5, Low EV Adoption",
    1: "Cluster 1 - Moderate Improvement PM2.5, Moderate EV Adoption",
    2: "Cluster 2 - Improving PM2.5, High EV Adoption",
    3: "Cluster 3 - Low Improvement in PM2.5 with Rising EV Adoption",
    4: "Cluster 4 - Stable PM2.5, Slow EV Growth"
}

cluster_colors = {
    0: '#e8e8e8',  # Color for Cluster 0
    1: '#ace4e4',  # Color for Cluster 1
    2: '#5ac8c8',  # Color for Cluster 2
    3: '#dfb0d6',  # Color for Cluster 3
    4: '#a5add3'   # Color for Cluster 4
}

# Step 4: Visualize clusters on a Folium map
m = folium.Map(location=[37.8, -96], zoom_start=4)

for _, row in gdf.iterrows():
    cluster_id = row['Cluster']
    description = cluster_descriptions.get(cluster_id, "No description available")
    popup_content = f"""
    <div style="font-size: 12px;">
        <b>County:</b> {row['County']}<br>
        <b>Cluster:</b> {cluster_id} - {description}<br><br>

        <b>PM2.5 Levels Over Time:</b><br>
        &bull; 2010: {row['Mean PM2.5_2010_x']}<br>
        &bull; 2017: {row['Mean PM2.5_2017_x']}<br>
        &bull; 2023: {row['Mean PM2.5_2023_x']}<br><br>

        <b>EV Adoption Over Time:</b><br>
        &bull; 2010: {row['Total_EV_Vehicles_2010_y']}<br>
        &bull; 2017: {row['Total_EV_Vehicles_2017_y']}<br>
        &bull; 2023: {row['Total_EV_Vehicles_2023_y']}
    </div>
    """

    # Create a polygon layer for the county with a tooltip
    folium.GeoJson(
        row['geometry'],
        style_function=lambda feature, color=cluster_colors[cluster_id]: {
            'fillColor': color,
            'color': 'black',
            'weight': 0.5,
            'fillOpacity': 0.7,
        },
        tooltip=folium.Tooltip(f"Cluster: {cluster_id}")
    ).add_to(m)

    # Calculate the centroid for placing the marker
    centroid = row['geometry'].centroid
    marker_location = [centroid.y, centroid.x]

    # Add a marker with a popup at the centroid of the polygon
    folium.Marker(
        location=marker_location,
        popup=folium.Popup(popup_content, max_width=300)
    ).add_to(m)

# Display the map
m


In [ ]:
vehicle_pop.head(5)

In [ ]:
!wget https://raw.githubusercontent.com/sg2083/GIS_repo/main/Ev_vs_disel.jpg -O myimage.jpg
# Image('myimage.jpg')
display(Image(filename='myimage.jpg', width=500))

The study examines the impacts of electric vehicle (EV) adoption on PM2.5 levels in California under various scenarios. These scenarios simulate different levels of electrification of the state’s vehicle fleet, ranging from partial electrification to full electrification of on-road vehicles. The specific scenarios and their descriptions are as follows:

### Simulation Scenarios:

- **BASE**:  
  This is the baseline scenario, where no electrification occurs, and the default on-road vehicle population is used. It serves as the comparison point for all other simulations.

- **EV25, EV50, EV100**:  
  These scenarios represent different levels of EV adoption, where 25%, 50%, and 100% of internal combustion engine (ICE) on-road vehicles are replaced by electric vehicles (EVs). These simulations show the changes in PM2.5 levels based on the percentage of vehicles electrified.

- **EV25_LDB, EV50_LDB, EV100_LDB (2016 only)**:  
  These scenarios go a step further, considering the electrification of a wider range of vehicles, including all passenger cars, trucks, light commercial trucks, transit buses, and school buses, not just on-road vehicles. These scenarios only apply to the year 2016 and simulate the impact of electrifying 25%, 50%, and 100% of these vehicles on PM2.5 levels.

- **EV100_NOBTW, EV100_NOBW, EV100_NOTW**:  
  These are variations of the **EV100** scenario, with specific factors like **biofuels (BW)** and **tire wear (TW)** zeroed out. Each of these simulations isolates the impact of certain emission sources:
  - **EV100_NOBTW**: Excludes both biofuels (BW) and tire wear (TW).
  - **EV100_NOBW**: Excludes biofuels (BW) but includes tire wear (TW).
  - **EV100_NOTW**: Excludes tire wear (TW) but includes biofuels (BW).

- **EV100_NOROADDUST**:  
  This scenario focuses on **EV100** while eliminating road dust, biofuels, and tire wear. It represents a scenario where all vehicle emissions are removed, except for the electrified vehicle emissions, and provides a cleaner comparison of EV impact.

### Summary:
The simulations assess how different levels of EV adoption and the inclusion or exclusion of various emission sources (like biofuels, tire wear, road dust) influence the PM2.5 levels in California. The findings show that EV adoption leads to a reduction in PM2.5 levels, with the greatest reductions observed in areas with high vehicle emissions, such as Los Angeles and the Central Valley.


In [ ]:
!wget https://raw.githubusercontent.com/sg2083/GIS_repo/main/Ev_if_increased.jpg -O myimage1.jpg
# Image('myimage.jpg')
display(Image(filename='myimage1.jpg', width=700))

In [ ]:
def process_normal_vehicle_data(df, year):
    # Filter data by year and fuel types
    filtered_normal_df = df[(df['Data Year'] == year) &
                     (~df['Fuel Type'].isin(['Battery Electric (BEV)', 'Plug-in Hybrid (PHEV)']))]

    # Group by county and fuel type and sum the vehicle counts
    county_fuel_count = filtered_normal_df.groupby(['County', 'Fuel Type'])['Number of Vehicles'].sum().reset_index()

    # Pivot the table for better readability
    county_fuel_pivot = county_fuel_count.pivot(index='County', columns='Fuel Type', values='Number of Vehicles').reset_index()

    # Fill NaN values with 0
    county_fuel_pivot = county_fuel_pivot.fillna(0)

    return county_fuel_pivot

county_normal_vehicles_data_2023 = process_normal_vehicle_data(vehicle_pop, 2023)
county_normal_vehicles_data_2010 = process_normal_vehicle_data(vehicle_pop, 2010)
county_normal_vehicles_data_2017 = process_normal_vehicle_data(vehicle_pop, 2017)

In [ ]:
county_normal_vehicles_data_2023['Total_Normal_Vehicles_2023'] = (county_normal_vehicles_data_2023['Diesel'] +
                                              county_normal_vehicles_data_2023['Flex Fuel'] + county_normal_vehicles_data_2023['Gasoline'] + county_normal_vehicles_data_2023['Gasoline Hybrid'] + county_normal_vehicles_data_2023['Natural Gas'])

county_normal_vehicles_data_2010['Total_Normal_Vehicles_2010'] = (county_normal_vehicles_data_2010['Diesel'] +
                                              county_normal_vehicles_data_2010['Flex Fuel'] + county_normal_vehicles_data_2010['Gasoline'] + county_normal_vehicles_data_2010['Gasoline Hybrid'] + county_normal_vehicles_data_2010['Natural Gas'])

county_normal_vehicles_data_2017['Total_Normal_Vehicles_2017'] = (county_normal_vehicles_data_2017['Diesel'] +
                                              county_normal_vehicles_data_2017['Flex Fuel'] + county_normal_vehicles_data_2017['Gasoline'] + county_normal_vehicles_data_2017['Gasoline Hybrid'] + county_normal_vehicles_data_2017['Natural Gas'])

county_normal_fuel_combined = county_normal_vehicles_data_2023[['County', 'Total_Normal_Vehicles_2023']].merge(
    county_normal_vehicles_data_2010[['County', 'Total_Normal_Vehicles_2010']], on='County', how='left').merge(
    county_normal_vehicles_data_2017[['County', 'Total_Normal_Vehicles_2017']], on='County', how='left')

# Replace any remaining NaN values with 0
county_normal_fuel_combined = county_normal_fuel_combined.fillna(0)

# Convert only numeric columns to integers, excluding 'County'
normal_numeric_columns = county_normal_fuel_combined.select_dtypes(include=['float', 'int']).columns
county_normal_fuel_combined[normal_numeric_columns] = county_normal_fuel_combined[normal_numeric_columns].astype(int)

# Print the result
county_normal_fuel_combined[['Total_Normal_Vehicles_2010', 'Total_Normal_Vehicles_2017', 'Total_Normal_Vehicles_2023']].head(3)

In [ ]:
# Assuming `merged_ev_data` and `county_normal_fuel_combined` are already defined
# Merge the EV and normal vehicle data based on County
combined_vehicle_data = merged_ev_data.merge(
    county_normal_fuel_combined,
    left_on='County',
    right_on='County',
    how='left'
)

# Replace any remaining NaN values with 0 in the merged data
combined_vehicle_data = combined_vehicle_data.fillna(0)

# Convert any numeric columns in the normal vehicle data to integers, excluding 'County'
numeric_columns = combined_vehicle_data.select_dtypes(include=['float', 'int']).columns
combined_vehicle_data[numeric_columns] = combined_vehicle_data[numeric_columns].astype(int)

combined_vehicle_data['Total_Vehicles_2010'] = combined_vehicle_data['Total_EV_Vehicles_2010'] + combined_vehicle_data['Total_Normal_Vehicles_2010']
combined_vehicle_data['Total_Vehicles_2017'] = combined_vehicle_data['Total_EV_Vehicles_2017'] + combined_vehicle_data['Total_Normal_Vehicles_2017']
combined_vehicle_data['Total_Vehicles_2023'] = combined_vehicle_data['Total_EV_Vehicles_2023'] + combined_vehicle_data['Total_Normal_Vehicles_2023']

print(combined_vehicle_data.head(2))

In [ ]:
combined_vehicle_data['County'] = combined_vehicle_data['County'].str.upper().str.strip()
CAl['NAME'] = CAl['NAME'].str.upper().str.strip()

combined_gdf = CAl[['NAME', 'geometry']].merge(
    combined_vehicle_data, left_on='NAME', right_on='County', how='inner'
)

# Convert the result to a GeoDataFrame if it isn’t already
combined_gdf = gpd.GeoDataFrame(combined_gdf, geometry='geometry')

# Display a sample of the combined data to verify
print(combined_gdf.head(1))


In [ ]:

!wget -q -O elections_2022.xlsx https://github.com/sg2083/GIS_repo/raw/main/senate_2022.xlsx

elections_2022 = pd.read_excel('elections_2022.xlsx')
# Step 1: Filter for California state only
california_data = elections_2022[elections_2022['state'] == "CALIFORNIA"]

# Step 2: Separate Democrat and Republican votes, using pivot for easier comparison
# First, keep only necessary columns to simplify the data
ca_votes = california_data[["county_name", "candidate", "party_simplified", "candidatevotes"]]

# Pivot table to get votes by party per county
ca_pivot = ca_votes.pivot_table(
    index="county_name",
    columns="party_simplified",
    values="candidatevotes",
    aggfunc="sum"
).reset_index()

# Step 3: Calculate the winner and vote margin
# Define function to determine the winner and margin
def determine_winner(row):
    if row['DEMOCRAT'] > row['REPUBLICAN']:
        return "DEMOCRAT", row['DEMOCRAT'] - row['REPUBLICAN']
    elif row['REPUBLICAN'] > row['DEMOCRAT']:
        return "REPUBLICAN", row['REPUBLICAN'] - row['DEMOCRAT']
    else:
        return "TIE", 0

# Apply function to calculate winner and margin
ca_pivot[['Winner', 'Margin']] = ca_pivot.apply(lambda row: pd.Series(determine_winner(row)), axis=1)

# Step 4: Finalize the Data - keeping only required columns
final_results = ca_pivot[["county_name", "Winner", "Margin"]]

# Display the final DataFrame
print(final_results.head(5))

In [ ]:
# Merge shapefile with election results data
# We assume county_name in final_results matches the county names in the shapefile
CAl['NAME'] = CAl['NAME'].str.upper()
map_data = CAl.merge(final_results, left_on="NAME", right_on="county_name")

# Define color mapping based on the 'Winner' column
color_map = {"DEMOCRAT": "#0078d4",  # Light blue
             "REPUBLICAN": "#d13438"}  # Light red (salmon)
map_data['color'] = map_data['Winner'].map(color_map)


In [ ]:
# m = folium.Map(location=[37.45, -119.4179], zoom_start=5.5, tiles=None,
#                zoom_control=False, scrollWheelZoom=False, dragging=False, doubleClickZoom=False)

# # Add GeoJson layer with style and popup for each county
# folium.GeoJson(
#     map_data,
#     style_function=lambda feature: {
#         'fillColor': feature['properties']['color'],
#         'color': 'white',
#         'weight': 0.5,
#         'fillOpacity': 0.9,
#     },
#     tooltip=folium.GeoJsonTooltip(
#         fields=['county_name', 'Winner', 'Margin'],
#         aliases=["County:", "Winner:", "Margin:"],
#         localize=True
#     ),
#     popup=None,
#     highlight_function=None
# ).add_to(m)

# # Display the map
# m
# map_data.columns

In [ ]:
# Normalize color gradient for percentage range
def get_color_for_percentage(percentage, color_map):
    # Normalize the value to fall within 0 to 1 for color mapping
    norm = mcolors.Normalize(vmin=0, vmax=100)
    colormap = plt.cm.get_cmap(color_map)  # Choose color map for each percentage
    color = colormap(norm(percentage))  # Get RGBA color
    return mcolors.to_hex(color)  # Convert RGBA to hex for folium

# Calculate EV and Normal Vehicle Percentages if not done already
combined_gdf['EV_Percentage_2017'] = (combined_gdf['Total_EV_Vehicles_2017'] / combined_gdf['Total_Vehicles_2017']) * 100
combined_gdf['Normal_Percentage_2017'] = (combined_gdf['Total_Normal_Vehicles_2017'] / combined_gdf['Total_Vehicles_2017']) * 100
combined_gdf['NAME'] = combined_gdf['NAME'].str.upper().str.strip()
map_data['NAME'] = map_data['NAME'].str.upper().str.strip()

# Merge combined_gdf with map_data to bring in EV and Normal Vehicle Percentages
map_data = map_data.merge(
    combined_gdf[['NAME', 'EV_Percentage_2017', 'Normal_Percentage_2017', 'Site Latitude', 'Site Longitude']],
    left_on='NAME', right_on='NAME',
    how='inner',
)

# Define color mapping based on the 'Winner' column
color_map = {"DEMOCRAT": "#0078d4",  # Light blue
             "REPUBLICAN": "#d13438"}  # Light red (salmon)
map_data['color'] = map_data['Winner'].map(color_map)



In [ ]:

map_data.columns
# Normalize color gradient for percentage range
def get_color_for_percentage(percentage, color_map):
    # Normalize the value to fall within 0 to 1 for color mapping
    norm = mcolors.Normalize(vmin=0, vmax=100)
    colormap = plt.cm.get_cmap(color_map)  # Choose color map for each percentage
    color = colormap(norm(percentage))  # Get RGBA color
    return mcolors.to_hex(color)  # Convert RGBA to hex for folium
# Calculate EV and Normal Vehicle Percentages if not done already
combined_gdf['EV_Percentage'] = (combined_gdf['Total_EV_Vehicles_2023'] / combined_gdf['Total_Vehicles_2023']) * 100
combined_gdf['Normal_Percentage'] = (combined_gdf['Total_Normal_Vehicles_2023'] / combined_gdf['Total_Vehicles_2023']) * 100
combined_gdf['NAME'] = combined_gdf['NAME'].str.upper().str.strip()
combined_gdf['Increased_Percentage_since_2017_EV'] = ((combined_gdf['Total_EV_Vehicles_2023']-combined_gdf['Total_EV_Vehicles_2017'] ) / combined_gdf['Total_Vehicles_2023']) * 100
map_data['NAME'] = map_data['NAME'].str.upper().str.strip()

# Merge combined_gdf with map_data to bring in EV and Normal Vehicle Percentages
map_data = map_data.merge(
    combined_gdf[['NAME', 'EV_Percentage', 'Normal_Percentage','Increased_Percentage_since_2017_EV']],
    left_on='NAME', right_on='NAME',
    how='inner',
)

# Define color mapping based on the 'Winner' column
color_map = {"DEMOCRAT": "#0078d4",  # Light blue
             "REPUBLICAN": "#d13438"}  # Light red (salmon)
map_data['color'] = map_data['Winner'].map(color_map)

# Create a folium map
m = folium.Map(location=[37.45, -119.4179], zoom_start=5.5, tiles="cartodb positron")

# Add GeoJson layer with color for each county based on the 'Winner' column
folium.GeoJson(
    map_data,
    style_function=lambda feature: {
        'fillColor': feature['properties']['color'],
        'color': 'white',
        'weight': 0.3,
        'fillOpacity': 0.9,
    },
    # tooltip=folium.GeoJsonTooltip(
    #     fields=['county_name', 'Winner', 'Margin', 'EV_Percentage', 'Normal_Percentage'],
    #     aliases=["County:", "Winner:", "Margin:", "EV %:", "Normal %:"],
    #     localize=True,
    #     sticky=True,
    # ),
).add_to(m)

# Add concentric CircleMarkers for each county based on EV and Normal vehicle percentages
for _, row in map_data.iterrows():
    location = [row['Site Latitude'], row['Site Longitude']]
    ev_percentage = row['EV_Percentage']
    normal_percentage = row['Normal_Percentage']

    # Get colors for each percentage
    ev_color = get_color_for_percentage(ev_percentage, "Greens")  # Green shades for EV
    normal_color = get_color_for_percentage(normal_percentage, "Reds")  # Red shades for Normal

    # Outer circle for EV Percentage
    folium.CircleMarker(
        location=location,
        radius=10,  # Slightly larger radius for EV
        color=ev_color,
        fill=True,
        fill_color=ev_color,
        fill_opacity=0.6,
        tooltip=(
            f"<b>County:</b> {row['county_name']}<br>"
            f"<b>Winner:</b> {row['Winner']}<br>"
            f"<b>Margin:</b> {row['Margin']}%<br>"
            f"<b>EV Percentage:</b> {ev_percentage:.2f}%<br>"
            f"<b>Normal Percentage:</b> {normal_percentage:.2f}%<br>"
            f"<b>EV increase since 2017:</b>{row['Increased_Percentage_since_2017_EV']:.2f}%"
        ),
    ).add_to(m)

    # Inner circle for Normal Percentage
    folium.CircleMarker(
        location=location,
        radius=4,  # Smaller radius for Normal
        color=normal_color,
        fill=True,
        fill_color=normal_color,
        fill_opacity=0.9,
    ).add_to(m)

# Display the map
m
